
# 🎥 FASE 1 – Coleta de Comentários do YouTube  
**Projeto:** Análise de Sentimentos em Comentários de Vídeos  
**Etapa do CRISP-DM:** Data Acquisition  

Este notebook coleta comentários dos vídeos mais visualizados de um canal do YouTube, aplicando boas práticas de paginação, detecção de idioma e exportação em CSV.
    

In [ ]:

# ============================================================
# 1️⃣ Importação das bibliotecas
# ============================================================
from googleapiclient.discovery import build
import pandas as pd
import time, os
from dotenv import load_dotenv
from langdetect import detect, DetectorFactory
import matplotlib.pyplot as plt

# Garante reprodutibilidade na detecção de idioma
DetectorFactory.seed = 0
    

In [ ]:

# ============================================================
# 2️⃣ Configurações iniciais e conexão com a API
# ============================================================
load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")
channel_id = os.getenv("CHANNEL_ID")

youtube = build("youtube", "v3", developerKey=api_key)
print("✅ Conexão estabelecida com a API do YouTube!")
    

In [ ]:

# ============================================================
# 3️⃣ Função para extrair comentários com paginação e tratamento de erros
# ============================================================
def extrair_comentarios(video_id, max_comments=1000):
    comentarios, token = [], None
    try:
        while True:
            response = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,
                textFormat="plainText",
                pageToken=token
            ).execute()

            for item in response.get("items", []):
                texto = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
                comentarios.append(texto)

            token = response.get("nextPageToken")
            if not token or len(comentarios) >= max_comments:
                break
            time.sleep(0.5)
    except Exception as e:
        print(f"⚠️ Erro ao buscar comentários do vídeo {video_id}: {e}")
    return comentarios[:max_comments]
    

In [ ]:

# ============================================================
# 4️⃣ Coleta dos vídeos mais visualizados do canal
# ============================================================
def coletar_comentarios_canal(channel_id, max_videos=5):
    search = youtube.search().list(
        part="id,snippet",
        channelId=channel_id,
        maxResults=max_videos,
        order="viewCount",
        type="video"
    ).execute()

    dados = []
    for item in search["items"]:
        video_id = item["id"]["videoId"]
        titulo = item["snippet"]["title"]
        coments = extrair_comentarios(video_id)
        for c in coments:
            try:
                idioma = detect(c)
            except:
                idioma = "indefinido"
            dados.append({
                "video_id": video_id,
                "video_titulo": titulo,
                "comentario": c,
                "idioma": idioma,
                "data_coleta": pd.Timestamp.now().strftime("%Y-%m-%d")
            })
    return pd.DataFrame(dados)
    

In [ ]:

# ============================================================
# 5️⃣ Execução principal da coleta
# ============================================================
df = coletar_comentarios_canal(channel_id)
print("✅ Coleta concluída!")
print(f"Total de vídeos analisados: {df['video_id'].nunique()}")
print(f"Total de comentários coletados: {len(df)}")
    

In [ ]:

# ============================================================
# 6️⃣ Tratamento final e exportação
# ============================================================
df.drop_duplicates(inplace=True)
df.to_csv("comentarios_top5.csv", index=False, encoding="utf-8-sig")

print("📂 Arquivo exportado: comentarios_top5.csv")
print(f"Linhas salvas: {len(df)}")
    

In [ ]:

# ============================================================
# 7️⃣ EDA básica – estatísticas descritivas
# ============================================================
df['tamanho'] = df['comentario'].astype(str).str.len()
print("\n📊 Estatísticas básicas do tamanho dos comentários:")
print(df['tamanho'].describe())

plt.figure(figsize=(6,3))
df['tamanho'].plot(kind='hist', bins=30, color='steelblue', edgecolor='black')
plt.title("Distribuição do Tamanho dos Comentários")
plt.xlabel("Número de caracteres")
plt.ylabel("Frequência")
plt.show()
    


---
✅ **Notebook finalizado com sucesso!**  
Comentários coletados, limpos e salvos para uso na Fase 2 (Análise de Sentimentos).

Próxima etapa: aplicação de modelos de NLP (BERT em português) para classificar os comentários como positivos, negativos ou neutros.
    